Purpose of this Code is to calculate RWA for Corporate Loans:

* Prepare Corporate Loan Data based on PD and LGD scoring data (these are input data required) because corporate and Concentra loans have not been implemnted in production when this code is created

* Create some columns (e.g. Insured_Class, Years_to_Maturity) that are needed for RWA calculation

* Calculate RWA assuming DLGD Addon = 0 (again, because they are not implemented yet, do not have addon values readily available and it is not straightforward to code)

* Calculate RWA by using addon scalar, which is created based on EQB personal loans (this addon file is output from EQB personal loan RWA calculation)

# Import Package

In [1]:
import sys
 
# appending a path
sys.path.append('')
 
import os
import pandas as pd
import numpy as np
import pickle
# import SFR_PD_Recalibration_2023_Lib as tool
# import Concentra_SFR_Fit_for_Use_2024_Lib as lib
from matplotlib import pyplot as plt
import math

import Lib_RWA as Lib_RWA

del sys.modules['Lib_RWA']  # Remove the module from sys.modules to force a reload

import Lib_RWA as Lib_RWA

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>div.output_scroll { height: 44em; }</style>"))
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("<style>.container { width:100% !important; }</style>"))

C:\Users\georgez\AppData\Local\Temp\ipykernel_9064\3764690850.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [3]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# Inputs

In [4]:
#Snapshot Info
snapshot_num = 202212 # 20250412 George commented out
snapshot = str(snapshot_num) # 20250412 George commented out
snapshot_date = '2022-12-31' # 20250412 George commented out

# snapshot_num = 202412 # 20250412 George added
# snapshot = str(snapshot_num) # 20250412 George added
# snapshot_date = '2024-12-31' # 20250412 George added



In [5]:

#CAR 2023 Ch5 prescribed
correlation_residential_mortgages = 0.15
correlation_residential_mortgages_rental = 0.22

CMHC_pd = 0.0001
CMHC_lgd = 0.11 #according to the newly developed Sovereign LGD model

lgd_gen_floor = 0.1 #general floor by CAR 2023


In [6]:
# Columns to Keep

RWA_Info_Cols_to_Keep = ['Loan_Number','Insured_class','EAD','Advance_Amount','Years_to_maturity', 'corr_uninsured', 'RWA_standardized']

PD_Cols_to_Keep = ['Loan_Number','SL_Date','PD_Pre_MOC','PD_Post_MOC_Pre_Adj','PD_Post_MOC','MRS_Bin_PD','Insured_Ind','Alt_Prime_Indicator','RemainingPrincipal_Tot_Exp','RemainingPrincipal_Excl_Partner', 'Corporate_Applicant_Ind','Combo_Province_Metro_Override','Combo_Province_Metro_Override_WOE', 'Prior_24_Worse_Delinquent_Status_FMT_Adj', 'Prior_24_Worse_Delinquent_Status_FMT_Adj_WOE','Beacon_Avg_App_CoApp','Beacon_Avg_App_CoApp_WOE','BF_LTV_Tot_Exp_FSA_Dw_WF','Combo_LTV_Insured_Ind','Combo_LTV_Insured_Ind_WOE']

PD_Cols_for_Corp = ['AdvancedAmount_EQB_Exp','AdvancedAmount_Total_Exp','AdvancedAmt_Incl_Part','AdvancedAmt_Excl_Part','Remaining_Term'] 


LGD_Cols_to_Keep = ['Loan_Number','MRS_Bin_LGD','LR_Avg_LGD','Segment_Avg_LGD','Base_Line_LGD','LGD_DT_Adjusted','Model_LGD','Sub_Product','BF_Appr_Prov_Dw','Appraisal_Bin_WOE', 'Occupancy', 'Occupancy_WOE', 'Province_Foreclosure', 'Province_Foreclosure_WOE', 'BF_LTV_Incl_Parter_Incl_HELOC_FSA_Dw','LTV_Bin_WOE' ]


# Set Folders

In [7]:
# Get current working directory

current_dir = os.getcwd()  # 20250403 George added

# code_dir= "C:\\Users" +'\\' + username + '\\' + "Equitable Bank\\EQB-Concentra Fit for Use - Fit_for_Use Development - Fit_for_Use Development\\RWA\\code" # 20250403 George commented out Francesca/Joseph's code

code_dir = current_dir # 20250403 George added

if int(snapshot_num) == 202412:
    input_dir = code_dir + '\\..\\' + "Inputs" # 20250406 George added
    Output_dir = code_dir + '\\..\\' + "Outputs" #20250406 George added

if int(snapshot_num) == 202212:
    input_dir = code_dir + '\\..\\' + "Dec. 2022 RWA Inputs" #20250410 George added
    Output_dir = code_dir + '\\..\\' + "Replicated Dec 2022 Outputs" #20250410 George added

# Prepare Coporate Loan Data

## Load PD Scoring Data

In [8]:
# add new SFR PD model generated PD 

#new_pd_data = pd.read_csv( Active_dir +'\\'+'PARAM_Pred_PD'  + '.csv', low_memory=False) # 20250402 George Commented out 
new_pd_data = pd.read_pickle(input_dir + '\\Rating_July2020_to_Dec2024_EQB (from George 20250414).pkl') # 20250402 George added
new_pd_data_red = new_pd_data.loc[new_pd_data['SL_Date'] == snapshot_date]


In [9]:
# print columns in alphabetical order in the new_pd_data_red dataframe #20250402 George added

print(sorted(new_pd_data_red.columns)) #20250402 George added


['AIRB_PD_Risk_Rating', 'AdvancedAmount_EQB_Exp', 'AdvancedAmount_Total_Exp', 'AdvancedAmt_Excl_Part', 'AdvancedAmt_Incl_Part', 'Alt_Prime_Indicator', 'AppraisalValue_Inception', 'BF_LTV_Tot_Exp_FSA_Dw_WF', 'Beacon_Avg_App_CoApp', 'Beacon_Avg_App_CoApp_WOE', 'City', 'Combo_LTV_Insured_Ind', 'Combo_LTV_Insured_Ind_WOE', 'Combo_Province_Metro_Override', 'Combo_Province_Metro_Override_WOE', 'Corporate_Applicant_Ind', 'Delinquency_Status_Adj', 'Dwelling_Type', 'FSA', 'Funded_Date', 'Insured_Ind', 'LoanType', 'Loan_Number', 'Metro_Region_BF_FMT', 'PD_Post_MOC', 'PD_Pre_MOC', 'PD_scoring', 'Prior_24_Worse_Delinquent_Status_FMT_Adj', 'Prior_24_Worse_Delinquent_Status_FMT_Adj_WOE', 'Province', 'RemainingPrincipal_Excl_Partner', 'RemainingPrincipal_Tot_Exp', 'Remaining_Term', 'SL_Date', 'Sub_Product', 'log_odds']


In [10]:
# For columns PD_Post_MOC_Pre_Adj and PD_Pre_MOC, if not in the data set, add them with nan #20250402 George added

if 'PD_Post_MOC_Pre_Adj' not in new_pd_data_red.columns: #20250402 George added

    new_pd_data_red['PD_Post_MOC_Pre_Adj'] = np.nan #20250402 George added

if 'PD_Pre_MOC' not in new_pd_data_red.columns: #20250402 George added

    new_pd_data_red['PD_Pre_MOC'] = np.nan #20250402 George added


# Rename AIRB_PD_Risk_Rating to be MRS_Bin #20250402 George added

new_pd_data_red = new_pd_data_red.rename({'AIRB_PD_Risk_Rating':'MRS_Bin'}, axis='columns')  #20250402 George added

C:\Users\georgez\AppData\Local\Temp\ipykernel_9064\1440374422.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_pd_data_red['PD_Post_MOC_Pre_Adj'] = np.nan #20250402 George added


In [11]:
# Save Loan number list for Chenxi as she requested to check
with pd.ExcelWriter(Output_dir +'\\'+'eqb_loans_list_dec24.xlsx' )as writer:
    new_pd_data_red['Loan_Number'].to_excel(writer, sheet_name='loan_list', index=False)

In [12]:
# Group by MRS_Bin, count distinct Loan_Number and min and max of PD_Post_MOC #20250402 George added
print('***************Checking Default Loans *****************') #20250402 George added
print(new_pd_data_red.groupby(['MRS_Bin'], as_index=False).agg({'Loan_Number': pd.Series.nunique, 'PD_Post_MOC': [min, max]})) #20250402 George added



***************Checking Default Loans *****************
  MRS_Bin Loan_Number PD_Post_MOC          
              nunique         min       max
0     1.0       29984    0.002228  0.002228
1     2.0       15099    0.003622  0.003622
2     3.0        6171    0.007377  0.007377
3     4.0        3732    0.011499  0.011499
4     5.0        1655    0.019067  0.019067
5     6.0         954    0.034554  0.034554
6     7.0         762    0.107857  0.107857
7     8.0         151    0.629118  0.629118
8     9.0          86    1.000000  1.000000


In [13]:
# # Remove Dafault Loans

# # Remove rows with PD_Post_MOC <=0 or >=1 (defaults) #20250402 George added

# new_pd_data_red = new_pd_data_red.loc[(new_pd_data_red['PD_Post_MOC'] > 0) & (new_pd_data_red['PD_Post_MOC'] < 1)] #20250402 George added

# # Group by MRS_Bin, count distinct Loan_Number and min and max of PD_Post_MOC #20250402 George added

# print('***************After Removing Default Loans*****************') #20250402 George added

# print(new_pd_data_red.groupby(['MRS_Bin'], as_index=False).agg({'Loan_Number': pd.Series.nunique, 'PD_Post_MOC': [min, max]})) #20250402 George added

In [14]:
# Rename Columns

new_pd_data_red.rename({'MRS_Bin':'MRS_Bin_PD'}, axis = 'columns', inplace=True) #20250412 George added

In [15]:
# Group by SL_Date and Corporate_Applicant_Ind, count number of Loan_Number

print('***************Group by SL_Date and Corporate_Applicant_Ind*****************') #20250403 George added

print(new_pd_data_red.groupby(['SL_Date','Corporate_Applicant_Ind'], as_index=False).agg({'Loan_Number': pd.Series.nunique})) #20250403 George added

***************Group by SL_Date and Corporate_Applicant_Ind*****************
     SL_Date  Corporate_Applicant_Ind  Loan_Number
0 2022-12-31                      0.0        56816
1 2022-12-31                      1.0         1778


## Load LGD Scoring Data

In [16]:
# add new SFR LGD model generated LGD and apply LGD floor 

# 2023_11-04 is most recent one from Ben
#new_lgd_data = pd.read_csv(input_dir+'\Dec_2022_LGD_2023_11_04_Recalibrated_Model.csv', low_memory = False)

if snapshot_num == 202212:

    new_lgd_data = pd.read_pickle(input_dir + '\\eqb_lgd_scored_2022_12 (from Abhi 20250410).pkl') # 20250406 George added

if snapshot_num == 202412:

    new_lgd_data = pd.read_pickle(input_dir + '\\eqb_lgd_scored_2024_12 (from Abhi 20250404).pkl') # 20250406 George added


In [17]:
# print duplicated columns 

print(new_lgd_data.columns[new_lgd_data.columns.duplicated()].tolist()) #20250412 George added

# Remove duplicated columns 

new_lgd_data = new_lgd_data.loc[:,~new_lgd_data.columns.duplicated()] #20250412 George added

['LTV_Bin_WOE', 'Appraisal_Bin_WOE']


In [18]:
# print columns in alphabetical order in the new_lgd_data dataframe 

print(sorted(new_lgd_data.columns)) #20250402 George added

['Appraisal_Bin', 'Appraisal_Bin_WOE', 'BF_Appr_Prov_Dw', 'BF_LTV_Incl_Parter_Incl_HELOC_FSA_Dw', 'Default_Ind', 'Final_LGD', 'Foreclosure_Ind', 'Foreclosure_Ind_WOE', 'Insured_Ind', 'LR_Avg_LGD', 'LTV_Bin', 'LTV_Bin_WOE', 'LoanType', 'Loan_Number', 'MRS_Bin', 'Mapped_LGD', 'Occupancy', 'Occupancy_WOE', 'Pred_LGD', 'Province_Foreclosure', 'Province_Foreclosure_WOE', 'RemainingPrincipal_Excl_Partner', 'SL_Date', 'Sub_Product']


In [19]:
# Add columns LGD_DT_Adjusted, Segment_Avg_LGD and Base_Line_LGD if they are not in the data set #20250412 George added

if 'LGD_DT_Adjusted' not in new_lgd_data.columns: # 20250412 George added
    new_lgd_data['LGD_DT_Adjusted'] = np.nan  # 20250412 George added, the numbers are not the latest and this will not be used in the final result

if 'Segment_Avg_LGD' not in new_lgd_data.columns: # 20250412 George added, same treatment as RWA for Concentra
    new_lgd_data['Segment_Avg_LGD'] = new_lgd_data.loc[:,'Mapped_LGD']

if 'Base_Line_LGD' not in new_lgd_data.columns: # 20250412 George added, same treatment as RWA for Concentra
    new_lgd_data['Base_Line_LGD'] = new_lgd_data.loc[:,'Mapped_LGD']

In [20]:
# Rename Columns

new_lgd_data = new_lgd_data.rename({'MRS_Bin':'MRS_Bin_LGD','Final_LGD':'Model_LGD'}, axis = 'columns')


## Merge PD and LGD data for Corporate Loans

In [21]:
# join new_Pd_data_red and new_lgd_data on Loan_Number

PD_LGD_Merge = pd.merge(new_pd_data_red[PD_Cols_to_Keep + PD_Cols_for_Corp], new_lgd_data[LGD_Cols_to_Keep], how='outer', on=['Loan_Number'])

In [22]:

# Filter with Corporate_Applicant_Ind = 1 #20250404 George added

PD_LGD_Merge_Corp = PD_LGD_Merge.loc[PD_LGD_Merge['Corporate_Applicant_Ind'] == 1] #20250404 George added

# Group by Insured_Ind, MRS_Bin_PD and MRS_Bin_LGD, count number of Loan_Number and sum of RemainginPrincipal_Tot_Exp#20250404 George added

print('***************Corporate Loans Only*****************') #20250404 George added

print(PD_LGD_Merge_Corp.groupby(['Insured_Ind','MRS_Bin_PD','MRS_Bin_LGD'], as_index=False).agg({'Loan_Number': pd.Series.nunique, 'RemainingPrincipal_Tot_Exp': sum})) #20250404 George added


***************Corporate Loans Only*****************
    Insured_Ind  MRS_Bin_PD  MRS_Bin_LGD  Loan_Number  RemainingPrincipal_Tot_Exp
0   Not Insured         1.0            1          258                1.074040e+08
1   Not Insured         1.0            2          129                6.356211e+07
2   Not Insured         1.0            3           20                7.839329e+06
3   Not Insured         2.0            1          146                8.270169e+07
4   Not Insured         2.0            2          323                2.090713e+08
5   Not Insured         2.0            3          253                1.686391e+08
6   Not Insured         2.0            4           96                3.947935e+07
7   Not Insured         3.0            1           37                2.518032e+07
8   Not Insured         3.0            2          116                7.501327e+07
9   Not Insured         3.0            3          118                8.362035e+07
10  Not Insured         3.0            4     

In [23]:

# Save data to excel #20250404 George added

with pd.ExcelWriter(Output_dir +'\\'+'Corporate_Loans_Breakdown.xlsx' )as writer:

    PD_LGD_Merge_Corp.to_excel(writer, sheet_name='Corp_Loans', index=False) #20250404 George added

# Load Corporate Loan Data

In [24]:
# Load Corporate_Loans_Breakdown.xlsx, sheet = Corp_Loans

Corp_Loans = pd.read_excel(Output_dir + '\\' + 'Corporate_Loans_Breakdown.xlsx', sheet_name='Corp_Loans')

In [25]:
# describe Remainging_Term

Corp_Loans['Remaining_Term'].describe()

count    1727.000000
mean       16.178923
std        12.290556
min         0.000000
25%         7.000000
50%        12.000000
75%        24.000000
max        60.000000
Name: Remaining_Term, dtype: float64

In [26]:
# Set rwa_data as a copy
rwa_data = Corp_Loans.copy()



# Add Column Insured_class

In [27]:
# if Corporate_Applicant_Ind has only one value 1, then add a column Insured_class with value "Uninsured"

if rwa_data['Corporate_Applicant_Ind'].nunique() == 1 and rwa_data['Corporate_Applicant_Ind'].iloc[0] == 1:
    rwa_data['Insured_class'] = 'Uninsured'

# Add Column corr_uninsured

In [28]:
# rwa_data['corr_uninsured'] =rwa_data['Occupancy_Status'].map(lambda x:0.22 if x in ['Rented','Unknown'] else 0.15) # 20250412 George commented out
rwa_data['corr_uninsured'] =rwa_data['Occupancy'].map(lambda x:0.15 if x in ['owner occupied','Owner Occupied','Owner_Occupied','owner_occupied'] else 0.22) # 20250412 George added

In [29]:
# print columns in alphabetical order

print(sorted(rwa_data.columns.tolist()))



['AdvancedAmount_EQB_Exp', 'AdvancedAmount_Total_Exp', 'AdvancedAmt_Excl_Part', 'AdvancedAmt_Incl_Part', 'Alt_Prime_Indicator', 'Appraisal_Bin_WOE', 'BF_Appr_Prov_Dw', 'BF_LTV_Incl_Parter_Incl_HELOC_FSA_Dw', 'BF_LTV_Tot_Exp_FSA_Dw_WF', 'Base_Line_LGD', 'Beacon_Avg_App_CoApp', 'Beacon_Avg_App_CoApp_WOE', 'Combo_LTV_Insured_Ind', 'Combo_LTV_Insured_Ind_WOE', 'Combo_Province_Metro_Override', 'Combo_Province_Metro_Override_WOE', 'Corporate_Applicant_Ind', 'Insured_Ind', 'Insured_class', 'LGD_DT_Adjusted', 'LR_Avg_LGD', 'LTV_Bin_WOE', 'Loan_Number', 'MRS_Bin_LGD', 'MRS_Bin_PD', 'Model_LGD', 'Occupancy', 'Occupancy_WOE', 'PD_Post_MOC', 'PD_Post_MOC_Pre_Adj', 'PD_Pre_MOC', 'Prior_24_Worse_Delinquent_Status_FMT_Adj', 'Prior_24_Worse_Delinquent_Status_FMT_Adj_WOE', 'Province_Foreclosure', 'Province_Foreclosure_WOE', 'RemainingPrincipal_Excl_Partner', 'RemainingPrincipal_Tot_Exp', 'Remaining_Term', 'SL_Date', 'Segment_Avg_LGD', 'Sub_Product', 'corr_uninsured']


# Add Other columns 

In [30]:
# dt_adjust={1:0.016497,2:0.040137,3:0.104959,4:0.180642}   #20250412 George commented out the numbers are not the latest and this will not be used in the final result
# rwa_data['LGD_DT_Adjusted']=rwa_data['MRS_Bin_LGD'].map(dt_adjust) #20250412 George commented out the numbers are not the latest and this will not be used in the final result

# if LGD_DT_Adjusted is not in the columns, add it as NaN

if 'LGD_DT_Adjusted' not in rwa_data.columns: # 20250412 George added
    rwa_data['LGD_DT_Adjusted'] = np.nan  # 20250412 George added, the numbers are not the latest and this will not be used in the final result

rwa_data['Advance_Amount'] = rwa_data.loc[:,'AdvancedAmount_EQB_Exp']
rwa_data['Years_to_maturity'] = rwa_data.loc[:,'Remaining_Term']/12
rwa_data['EAD'] = rwa_data.loc[:,'RemainingPrincipal_Excl_Partner']

# RWA with 0 Addon

In [31]:
# Force Addon to be 0

rwa_raw_data_5 = rwa_data.copy()
rwa_raw_data_5['AddOn']=0


In [32]:
# Run RWA calculation function

[df_out, rwa_by_Insured_class, rwa_by_MRS_Bin, res_data]  = Lib_RWA.rwa_calculation(rwa_raw_data_5, lgd_gen_floor, CMHC_lgd, CMHC_pd)



0
1
2


In [33]:
# print snapshot date 

print('Snapshot date: ' + snapshot_date) #20250403 George added

# print the shape of res_data

print('Shape of res_data: ' + str(res_data.shape)) #20250403 George added

# Group by Insured_Indicator, count number of Loan_Number, sum of EAD and RWA_new

print('***************Group by Insured_Indicator*****************') #20250403 George added

Sumry = res_data.groupby(['Insured_class'], as_index=False).agg({'Loan_Number': pd.Series.nunique, 'EAD': sum, 'RWA_new': sum}) #20250403 George added

# Add a total row to Sumry

Sumry = Sumry.append(Sumry.sum(numeric_only=True), ignore_index=True) #20250403 George added
Sumry.at[Sumry.index[-1], 'Insured_class'] = 'Total' #20250403 George added

# Add a column RWA_Ratio as the ratio of RWA_new and EAD

Sumry['RWA_Ratio'] = Sumry['RWA_new'] / Sumry['EAD'] #20250403 George added

# print the summary

print(Sumry) #20250403 George added

# print columns in alphabetical ored

print(sorted(res_data.columns)) #20250403 George added

Snapshot date: 2022-12-31
Shape of res_data: (1778, 58)
***************Group by Insured_Indicator*****************
  Insured_class  Loan_Number           EAD       RWA_new  RWA_Ratio
0     Uninsured       1778.0  1.027965e+09  2.811028e+08   0.273456
1         Total       1778.0  1.027965e+09  2.811028e+08   0.273456
['AddOn', 'Advance_Amount', 'AdvancedAmount_EQB_Exp', 'AdvancedAmount_Total_Exp', 'AdvancedAmt_Excl_Part', 'AdvancedAmt_Incl_Part', 'Alt_Prime_Indicator', 'Appraisal_Bin_WOE', 'BF_Appr_Prov_Dw', 'BF_LTV_Incl_Parter_Incl_HELOC_FSA_Dw', 'BF_LTV_Tot_Exp_FSA_Dw_WF', 'Base_Line_LGD', 'Beacon_Avg_App_CoApp', 'Beacon_Avg_App_CoApp_WOE', 'CalibratedPD', 'Combo_LTV_Insured_Ind', 'Combo_LTV_Insured_Ind_WOE', 'Combo_Province_Metro_Override', 'Combo_Province_Metro_Override_WOE', 'Corporate_Applicant_Ind', 'DLGD_floor', 'EAD', 'EAD_DLGD_Modified', 'EAD_Post_CRM', 'Final_LGD', 'Insured_Ind', 'Insured_class', 'LGD_Before_Floors', 'LGD_DT_Adjusted', 'LR_Avg_LGD', 'LTV_Bin_WOE', 'Loan_Numb

C:\Users\georgez\AppData\Local\Temp\ipykernel_9064\1574102103.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Sumry = Sumry.append(Sumry.sum(numeric_only=True), ignore_index=True) #20250403 George added


In [34]:
# Save Data
     
with pd.ExcelWriter(
   Output_dir + '\concentra_rwa_addon_zero.xlsx') as writer: #20250406 George added
          
     res_data.to_excel(writer, sheet_name="RWA_Data")

# Borrow Add on Impact from EQB

In [35]:
# We need to add a cross table of lgd_bins and PD bins for rwa with add-on. However, we don't have account level addon info. therefore we take the impact from EQB at the Combo_LTV_Insured_Ind X Combo_Province_Metro_Override_WOE level

# addon_impact =pd.read_excel(r"C:\Users\francescav\OneDrive - Equitable Bank\EQB-Concentra Fit for Use - Fit_for_Use Development - Fit_for_Use Development\RWA\code\Outputs\addon_impact.xlsx")  # 20250406 George commented out Francesca's code

addon_impact =pd.read_excel(Output_dir+r"\addon_impact.xlsx") # 20250406 George added

rwa_raw_data = res_data.copy() # 20250411 George added because a new res_data is created in the above function

In [36]:
# 20250403 - Francesca Add on already has mapped values !
addon_impact.Combo_Province_Metro_Override_WOE.value_counts()

-0.389936    7
-0.161658    7
 0.165650    7
 0.253031    6
Name: Combo_Province_Metro_Override_WOE, dtype: int64

In [37]:
addon_impact.Combo_Province_Metro_Override_WOE.fillna(0, inplace=True)
addon_impact.Combo_Province_Metro_Override_WOE =addon_impact.Combo_Province_Metro_Override_WOE.round(8)

In [38]:
addon_impact.rename(columns={'impact':'addon_impact'},inplace=True)

In [39]:
#process to make sure the merge works;
rwa_raw_data.Combo_Province_Metro_Override_WOE=rwa_raw_data.Combo_Province_Metro_Override_WOE.round(6).fillna(0)
addon_impact.Combo_Province_Metro_Override_WOE = addon_impact.Combo_Province_Metro_Override_WOE.round(6).fillna(0)
addon_impact.Combo_LTV_Insured_Ind=addon_impact.Combo_LTV_Insured_Ind.fillna('')

In [40]:
rwa_raw_data_addon=rwa_raw_data.merge(addon_impact[['Combo_LTV_Insured_Ind','Combo_Province_Metro_Override_WOE','addon_impact']], on=['Combo_LTV_Insured_Ind','Combo_Province_Metro_Override_WOE'],how='left')

In [41]:
rwa_raw_data_addon.addon_impact = rwa_raw_data_addon.addon_impact.fillna(1)

In [42]:
rwa_raw_data_addon['RWA_AddOn']=rwa_raw_data_addon['RWA_new']*rwa_raw_data_addon['addon_impact']

# Sanity Check of Outputs

In [48]:
# print snapshot date 

print('Snapshot date: ' + snapshot_date) #20250403 George added

# print the shape of rwa_raw_data_addon

print('Shape of rwa_raw_data_addon: ' + str(rwa_raw_data_addon.shape)) #20250403 George added

# Group by Insured_Indicator, count number of Loan_Number, sum of EAD and RWA_AddOn

print('***************Group by Insured_Indicator*****************') #20250403 George added

Sumry = rwa_raw_data_addon.groupby(['Insured_class'], as_index=False).agg({'Loan_Number': pd.Series.nunique, 'EAD': sum, 'RWA_AddOn': sum}) #20250403 George added

# Add a total row to Sumry

Sumry = Sumry.append(Sumry.sum(numeric_only=True), ignore_index=True) #20250403 George added
Sumry.at[Sumry.index[-1], 'Insured_class'] = 'Total' #20250403 George added

# Add a column RWA_Ratio as the ratio of RWA_AddOn and EAD

Sumry['RWA_Ratio'] = Sumry['RWA_AddOn'] / Sumry['EAD'] #20250403 George added

# print the summary

print(Sumry) #20250403 George added

# print columns in alphabetical ored

print(sorted(rwa_raw_data_addon.columns)) #20250403 George added

Snapshot date: 2022-12-31
Shape of rwa_raw_data_addon: (1778, 60)
***************Group by Insured_Indicator*****************
  Insured_class  Loan_Number           EAD     RWA_AddOn  RWA_Ratio
0     Uninsured       1778.0  1.027965e+09  3.877582e+08   0.377209
1         Total       1778.0  1.027965e+09  3.877582e+08   0.377209
['AddOn', 'Advance_Amount', 'AdvancedAmount_EQB_Exp', 'AdvancedAmount_Total_Exp', 'AdvancedAmt_Excl_Part', 'AdvancedAmt_Incl_Part', 'Alt_Prime_Indicator', 'Appraisal_Bin_WOE', 'BF_Appr_Prov_Dw', 'BF_LTV_Incl_Parter_Incl_HELOC_FSA_Dw', 'BF_LTV_Tot_Exp_FSA_Dw_WF', 'Base_Line_LGD', 'Beacon_Avg_App_CoApp', 'Beacon_Avg_App_CoApp_WOE', 'CalibratedPD', 'Combo_LTV_Insured_Ind', 'Combo_LTV_Insured_Ind_WOE', 'Combo_Province_Metro_Override', 'Combo_Province_Metro_Override_WOE', 'Corporate_Applicant_Ind', 'DLGD_floor', 'EAD', 'EAD_DLGD_Modified', 'EAD_Post_CRM', 'Final_LGD', 'Insured_Ind', 'Insured_class', 'LGD_Before_Floors', 'LGD_DT_Adjusted', 'LR_Avg_LGD', 'LTV_Bin_WOE', 

C:\Users\georgez\AppData\Local\Temp\ipykernel_9064\601514633.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Sumry = Sumry.append(Sumry.sum(numeric_only=True), ignore_index=True) #20250403 George added


In [44]:
# Create a pivot table, with rows as MRS_Bin_PD, columns as MRS_Bin_LGD, and values as count of Loan_Number, sum of EAD and RWA_AddOn, ratio of RWA_AddOn and EAD

rwa_raw_data_addon['MRS_Bin_PD'] = rwa_raw_data_addon['MRS_Bin_PD'].astype(str) #20250403 George added

rwa_raw_data_addon['MRS_Bin_LGD'] = rwa_raw_data_addon['MRS_Bin_LGD'].astype(str) #20250403 George added

pivot_table = rwa_raw_data_addon.pivot_table(index='MRS_Bin_PD', columns='MRS_Bin_LGD', values=['Loan_Number', 'EAD', 'RWA_AddOn'], aggfunc={'Loan_Number': pd.Series.nunique, 'EAD': sum, 'RWA_AddOn': sum})


# show the numbers as comma separated

pd.options.display.float_format = '{:,.0f}'.format 

# print the pivot table

print(pivot_table) 

# restore option display to default

pd.reset_option('display.float_format')

                    EAD                                    Loan_Number             RWA_AddOn                                 
MRS_Bin_LGD           1           2           3          4           1   2   3   4         1          2          3          4
MRS_Bin_PD                                                                                                                   
1           107,377,899  63,392,041   7,839,329        NaN         258 129  20 NaN 7,127,338  8,124,596  1,452,903        NaN
2            82,570,921 208,835,452 168,390,879 39,427,811         146 323 253  96 7,756,674 47,979,751 59,955,990 27,415,881
3            25,131,764  74,721,145  83,531,656 28,125,546          37 116 118  60 3,633,898 26,878,050 48,933,224 32,343,019
4            11,527,962  20,047,280  36,430,509 12,053,340          16  35  48  29 2,436,943  8,934,270 27,857,767 18,004,984
5             7,850,199   7,230,992  12,085,766  2,123,470           8  13  16   9 2,020,997  3,709,893 12,866,006  3,

# Save Data

In [45]:
# Save Data

with open( Output_dir +r'\EQB_Corp_rwa_addon.pkl', 'wb') as file: # 20250406 George added

    # A new file will be created 
    pickle.dump(rwa_raw_data_addon, file) 

In [46]:
#Save an excel copy

rwa_raw_data_addon.to_excel(Output_dir + r'\EQB_Corp_rwa_addon.xlsx')  #20250406 George added